In [7]:
from sklearn.neighbors import KNeighborsClassifier

import os
os.chdir('/Users/kshitijphulare/DLH-Proj/')
from pre_processing import *
from main import *
# from dataset.preprocessing.tf_idf_all_feature_matrix_gen import TFIDFFeatureGeneration
from sklearn.metrics import f1_score
import numpy as np
from sklearn.model_selection import KFold
import csv
import pandas as pd

In [8]:
class KNN:
    def __init__(self, x_train, y_train, x_test, y_test, n):
        self.knn = KNeighborsClassifier(n_neighbors=n)
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test

    def train(self):
        self.knn.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.knn.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        return f1_macro, f1_micro

In [11]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous_Insufficiency']
column_headings = ["Morbidity Class", "KNN1_Macro F1", "KNN1_Micro F1", "KNN5_Macro F1", "KNN5_Micro F1"]

with open("./performance_KNN.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2], column_headings[3], column_headings[4]])
total_macro1=[]
total_micro1= []
total_macro5=[]
total_micro5= []

In [12]:
for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = tf_idf(train_preprocessed_df, morbidity)
    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)
    

    f1_macro_list1 = []
    f1_micro_list1 = []
    f1_macro_list5 = []
    f1_micro_list5 = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training KNN using TF-IDF Representation
        knn1_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 1)
        knn1_obj.train()

        f1_macro1, f1_micro1 = knn1_obj.test_and_evaluate()

        f1_macro_list1.append(f1_macro1)
        f1_micro_list1.append(f1_micro1)

        knn5_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 5)
        knn5_obj.train()

        f1_macro5, f1_micro5 = knn5_obj.test_and_evaluate()

        f1_macro_list5.append(f1_macro5)
        f1_micro_list5.append(f1_micro5)

    f1_macro1 = np.mean(f1_macro_list1)
    f1_micro1 = np.mean(f1_micro_list1)
    f1_macro5 = np.mean(f1_macro_list5)
    f1_micro5 = np.mean(f1_micro_list5)

    print(f"For n=1, Macro F1 score: {f1_macro1} and Micro F1 Score {f1_micro1}")
    print(f"For n=5, Macro F1 score: {f1_macro5} and Micro F1 Score {f1_micro5}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro1, f1_micro1, f1_macro5, f1_micro5]
    total_macro1.append(f1_macro1)
    total_micro1.append(f1_micro1)
    total_macro5.append(f1_macro5)
    total_micro5.append(f1_micro5)
    with open("./performance_KNN.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)
        
with open("./performance_KNN.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ['Average']
    row.extend([sum(total_macro1)/len(total_macro1),sum(total_micro1)/len(total_micro1),sum(total_macro5)/len(total_macro5),sum(total_micro5)/len(total_micro5)])
    writer.writerow(row)

Asthma
(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
For n=1, Macro F1 score: 0.5394919208958887 and Micro F1 Score 0.7884150030248034
For n=5, Macro F1 score: 0.5183582141185823 and Micro F1 Score 0.8741379310344828
CAD
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
For n=1, Macro F1 score: 0.6398212373158091 and Micro F1 Score 0.6787878787878788
For n=5, Macro F1 score: 0.6242912372674849 and Micro F1 Score 0.6823905723905723
CHF
(243, 600) (243,) Counter({1.0: 243})
For n=1, Macro F1 score: 1.0 and Micro F1 Score 1.0
For n=5, Macro F1 score: 1.0 and Micro F1 Score 1.0
Depression
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
For n=1, Macro F1 score: 0.5954536446280783 and Micro F1 Score 0.724926943308007
For n=5, Macro F1 score: 0.5801692139079798 and Micro F1 Score 0.7799532437171244
Diabetes
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
For n=1, Macro F1 score: 0.554123683743071 and Micro F1 Score 0.6544172932330827
For n=5, Macro F1 score: 0.5482992726097935 and Micro F1 Sco